LSTM trained on gridded forcings for each station

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../..')
import os
import gc
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
from sklearn import preprocessing
import netCDF4 as nc
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from src import load_data, evaluate, datasets, utils
import torch.autograd as autograd

time_stamp = datetime.now().strftime('%Y%m%d-%H%M%S')
time_stamp

'20190906-083049'

In [2]:
USE_CUDA = False
if torch.cuda.is_available():
    print('CUDA Available')
    USE_CUDA = True
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
device = torch.device('cuda:0' if USE_CUDA else 'cpu')
num_devices = torch.cuda.device_count() if USE_CUDA else 0
print('cuda devices: {}'.format(list(torch.cuda.get_device_name(i) for i in range(num_devices))))
torch.manual_seed(0)
np.random.seed(0)

writer = SummaryWriter()

CUDA Available
cuda devices: ['Tesla V100-SXM2-16GB']


In [3]:
rdrs_vars = list(range(8))
agg = None
seq_len = 5*24
seq_steps = 1
validation_fraction = 0.1
learning_rate = 2e-3
patience = 150
min_improvement = 0.01
batch_size = 6
H = 20
lstm_layers = 1
epochs = 800
train_start = datetime.strptime('2010-01-01', '%Y-%m-%d') + timedelta(hours=seq_len * seq_steps)
train_end = '2012-12-31'
test_start = '2013-01-01'
test_end = '2014-12-31'

In [4]:
# For each station, read which grid cells belong to its subwatershed
station_cell_mapping = load_data.get_station_cell_mapping()

In [5]:
def get_datasets(station):
    train_dataset = datasets.RdrsDataset(rdrs_vars, seq_len, seq_steps, train_start, train_end, include_months=True, aggregate_daily=agg, station=station)
    test_dataset = datasets.RdrsDataset(rdrs_vars, seq_len, seq_steps, test_start, test_end, include_months=True, aggregate_daily=agg,
                                        conv_scalers=train_dataset.conv_scalers, fc_scalers=train_dataset.fc_scalers, station=station)

    val_indices = np.random.choice(len(train_dataset), size=int(validation_fraction * len(train_dataset)), replace=False)
    train_indices = list(i for i in range(len(train_dataset)) if i not in val_indices)
    train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, sampler=train_sampler, pin_memory=True, drop_last=False)
    val_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, sampler=val_sampler, pin_memory=True, drop_last=False)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=False, pin_memory=True, drop_last=False)
    
    return train_dataloader, val_dataloader, test_dataloader, test_dataset.data_runoff.copy(), train_dataset.x_conv.shape, train_dataset.fc_var_names

In [6]:
class LSTMRegression(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, batch_size):
        super(LSTMRegression, self).__init__()
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers)
        self.linear = nn.Linear(hidden_dim, 1)
        self.init_hidden(batch_size)
    def init_hidden(self, batch_size):
        self.hidden = (torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=device, requires_grad=True),
                       torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=device, requires_grad=True))

    def forward(self, input):
        lstm_out, self.hidden = self.lstm(input.permute(1,0,2), self.hidden)
        return self.linear(lstm_out[-1])

In [7]:
predictions = {}
actuals = {}
models = {}

for station in station_cell_mapping['station'].unique():
    print(station)
    train_dataloader, val_dataloader, test_dataloader, test_streamflow, x_conv_shape, fc_varnames = get_datasets(station)
    onehot_vars = list(i for i,v in enumerate(fc_varnames) if v.startswith('month'))
    
    station_cell_mask = torch.zeros(x_conv_shape[-2:]).bool()
    for _, row in station_cell_mapping[station_cell_mapping['station'] == station][['row', 'col']].iterrows():
        station_cell_mask[row['col'], row['row']] = True
    
    best_loss_model = (-1, np.inf, None)
    model = LSTMRegression(x_conv_shape[2] * int(station_cell_mask.sum()) + len(onehot_vars), H, lstm_layers, batch_size).to(device)
    loss_fn = evaluate.NSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(epochs):
        model.train()
        train_losses = torch.tensor(0.0)
        for i, train_batch in enumerate(train_dataloader):
            x_train = train_batch['x_conv'][...,station_cell_mask].reshape(*train_batch['x_conv'].shape[:2], -1)
            x_train = torch.cat([x_train, train_batch['x_fc'][:,onehot_vars].unsqueeze(dim=1).repeat(1,x_train.shape[1],1)], dim=2).to(device)
            model.init_hidden(train_batch['x_conv'].shape[0])
            y_pred = model(x_train)
            
            train_loss = loss_fn(y_pred.reshape(-1), train_batch['y'].to(device), means=train_batch['y_mean'].to(device))
            
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            
            train_losses += train_loss.detach()
            
        train_loss = (train_losses / len(train_dataloader)).item()
        print('Epoch', epoch, 'mean loss:', train_loss)
        writer.add_scalar('loss_' + station, train_loss, epoch)
        
        model.eval()
        val_losses = torch.tensor(0.0)
        for i, val_batch in enumerate(val_dataloader):
            x_val = val_batch['x_conv'][...,station_cell_mask].reshape(*val_batch['x_conv'].shape[:2], -1)
            x_val = torch.cat([x_val, val_batch['x_fc'][:,onehot_vars].unsqueeze(dim=1).repeat(1,x_val.shape[1],1)], dim=2).to(device)
            model.init_hidden(val_batch['x_conv'].shape[0])
            y_pred = model(x_val)

            loss = loss_fn(y_pred.reshape(-1), val_batch['y'].to(device), means=val_batch['y_mean'].to(device))
            val_losses += loss.detach()
        
        val_loss = (val_losses / len(val_dataloader)).item()
        print('Epoch', epoch, 'mean val loss:', val_loss)
        writer.add_scalar('loss_val_' + station, val_loss, epoch)
        if val_loss < best_loss_model[1] - min_improvement:
            best_loss_model = (epoch, val_loss, model.state_dict())  # new best model
            load_data.pickle_model('LSTM_VIC', model, station, time_stamp)
        elif epoch > best_loss_model[0] + patience:
            print('Patience exhausted in epoch {}. Best loss was {}'.format(epoch, best_loss_model[1]))
            break
    
    del train_dataloader, val_dataloader, x_conv_shape, fc_varnames
    print('Using best model from epoch', str(best_loss_model[0]), 'which had loss', str(best_loss_model[1]))
    model.load_state_dict(best_loss_model[2])
    model.eval()
    predict = test_streamflow.copy()
    predict['runoff'] = np.nan
    pred_array = np.array([])
    for i, test_batch in enumerate(test_dataloader):
        x_test = test_batch['x_conv'][...,station_cell_mask].reshape(*test_batch['x_conv'].shape[:2], -1)
        x_test = torch.cat([x_test, test_batch['x_fc'][:,onehot_vars].unsqueeze(dim=1).repeat(1,x_test.shape[1],1)], dim=2).to(device)
        model.init_hidden(test_batch['x_conv'].shape[0])
        pred_array = np.concatenate([pred_array, model(x_test).detach().cpu().numpy().reshape(-1)])
    del test_dataloader
    predict['runoff'] = pred_array
    predictions[station] = predict.set_index('date')
    actuals[station] = test_streamflow.set_index('date')['runoff']
    models[station] = model
    gc.collect()

02GA010
Epoch 0 mean loss: 1.3051220178604126
Epoch 0 mean val loss: 0.9630147218704224
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_02GA010_20190906-083049.pkl


/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type LSTMRegression. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch 1 mean loss: 0.7682093977928162
Epoch 1 mean val loss: 1.8573664426803589
Epoch 2 mean loss: 0.7437446117401123
Epoch 2 mean val loss: 0.8112610578536987
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_02GA010_20190906-083049.pkl
Epoch 3 mean loss: 0.7112101316452026
Epoch 3 mean val loss: 0.9600921273231506
Epoch 4 mean loss: 0.6470354795455933
Epoch 4 mean val loss: 0.7105724215507507
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_02GA010_20190906-083049.pkl
Epoch 5 mean loss: 0.6410133838653564
Epoch 5 mean val loss: 0.7643378973007202
Epoch 6 mean loss: 0.6454828381538391
Epoch 6 mean val loss: 0.7019945979118347
Epoch 7 mean loss: 0.6119781136512756
Epoch 7 mean val loss: 0.8319128155708313
Epoch 8 mean loss: 0.6008170247077942
Epoch 8 mean val loss: 0.6771731376647949
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_02GA010_20190906-083049.pkl
Epoch 9 mean loss: 0.5657998323440552
Epoch 9 mean val loss: 0.681304752

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type LSTMRegression. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch 1 mean loss: 0.7150851488113403
Epoch 1 mean val loss: 0.6854627728462219
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_02GA018_20190906-083049.pkl
Epoch 2 mean loss: 0.6995723247528076
Epoch 2 mean val loss: 0.9937911033630371
Epoch 3 mean loss: 0.6924697756767273
Epoch 3 mean val loss: 0.6368004083633423
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_02GA018_20190906-083049.pkl
Epoch 4 mean loss: 0.6468368172645569
Epoch 4 mean val loss: 0.6308887600898743
Epoch 5 mean loss: 0.6044715046882629
Epoch 5 mean val loss: 0.6199495196342468
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_02GA018_20190906-083049.pkl
Epoch 6 mean loss: 0.5810690522193909
Epoch 6 mean val loss: 0.6515883803367615
Epoch 7 mean loss: 0.5824149250984192
Epoch 7 mean val loss: 0.5479258298873901
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_02GA018_20190906-083049.pkl
Epoch 8 mean loss: 0.5808388590812683
Epoch 8 mean val l

...

Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04214500_20190906-083049.pkl
Epoch 1 mean loss: 1.047853708267212
Epoch 1 mean val loss: 1.8581959009170532
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04214500_20190906-083049.pkl
Epoch 2 mean loss: 0.7806076407432556
Epoch 2 mean val loss: 1.719550371170044
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04214500_20190906-083049.pkl
Epoch 3 mean loss: 0.6757704615592957
Epoch 3 mean val loss: 1.5530753135681152
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04214500_20190906-083049.pkl
Epoch 4 mean loss: 0.6890581846237183
Epoch 4 mean val loss: 1.533271312713623
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04214500_20190906-083049.pkl
Epoch 5 mean loss: 0.5859265923500061
Epoch 5 mean val loss: 1.7972736358642578
Epoch 6 mean loss: 0.5075548887252808
Epoch 6 mean val loss: 1.332202672958374
Saved model as /home/mgauch/runoff-nn/s

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type LSTMRegression. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04215000_20190906-083049.pkl
Epoch 1 mean loss: 0.6302992701530457
Epoch 1 mean val loss: 0.7200091481208801
Epoch 2 mean loss: 0.594646155834198
Epoch 2 mean val loss: 0.6053547859191895
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04215000_20190906-083049.pkl
Epoch 3 mean loss: 0.5496243238449097
Epoch 3 mean val loss: 0.5631301999092102
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04215000_20190906-083049.pkl
Epoch 4 mean loss: 0.500066876411438
Epoch 4 mean val loss: 0.511465311050415
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04215000_20190906-083049.pkl
Epoch 5 mean loss: 0.49598103761672974
Epoch 5 mean val loss: 0.5263069272041321
Epoch 6 mean loss: 0.4645857512950897
Epoch 6 mean val loss: 0.538001298904419
Epoch 7 mean loss: 0.46273109316825867
Epoch 7 mean val loss: 0.4969885051250458
Saved model as /home/mgauch/runoff-nn/src/../pickle/mo

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type LSTMRegression. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04215500_20190906-083049.pkl
Epoch 1 mean loss: 0.6768521666526794
Epoch 1 mean val loss: 0.5228666663169861
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04215500_20190906-083049.pkl
Epoch 2 mean loss: 0.6476098299026489
Epoch 2 mean val loss: 0.5018365979194641
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04215500_20190906-083049.pkl
Epoch 3 mean loss: 0.536793053150177
Epoch 3 mean val loss: 0.4965254068374634
Epoch 4 mean loss: 0.5237445831298828
Epoch 4 mean val loss: 0.40878257155418396
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04215500_20190906-083049.pkl
Epoch 5 mean loss: 0.5432793498039246
Epoch 5 mean val loss: 0.4085315465927124
Epoch 6 mean loss: 0.4578567147254944
Epoch 6 mean val loss: 0.33856961131095886
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04215500_20190906-083049.pkl
Epoch 7 mean loss: 0.4497186243534

In [8]:
nse_list = []
mse_list = []
for station, predict in predictions.items():
    nse, mse = evaluate.evaluate_daily(station, predict['runoff'], actuals[station], writer=writer)
    nse_list.append(nse)
    mse_list.append(mse)
    
    print(station, '\tNSE:', nse, '\tMSE:', mse, '(clipped to 0)')

result_str = 'Median NSE (clipped to 0) {} / Min {} / Max {}'.format(np.median(nse_list), np.min(nse_list), np.max(nse_list))
print(result_str)
print('Median MSE (clipped to 0)', np.median(mse_list), '/ Min', np.min(mse_list), '/ Max', np.max(mse_list))
writer.add_text('Results', result_str)

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/pandas/plotting/_converter.py:129: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


02GA010 	NSE: 0.016078140995981705 	MSE: 469.10791593672593 (clipped to 0)
02GA018 	NSE: 0.1275999083987377 	MSE: 219.1060576759262 (clipped to 0)
02GA038 	NSE: 0.051845830553763794 	MSE: 156.00782397143692 (clipped to 0)
02GA047 	NSE: 0.03017389246579749 	MSE: 76.02469249003414 (clipped to 0)
02GB001 	NSE: 0.01656499250431087 	MSE: 7385.474172229335 (clipped to 0)
02GB007 	NSE: 0.28219904896079173 	MSE: 22.44708176962875 (clipped to 0)
02GC002 	NSE: 0.28721658410802664 	MSE: 91.59483196927968 (clipped to 0)
02GC007 	NSE: 0.17725091232797585 	MSE: 24.782308685282814 (clipped to 0)
02GC010 	NSE: 0.24246024251685883 	MSE: 45.035677826826635 (clipped to 0)
02GC018 	NSE: 0.3153934002077937 	MSE: 46.366215443138806 (clipped to 0)
02GC026 	NSE: 0.2539358440713346 	MSE: 119.99155236748412 (clipped to 0)
02GD004 	NSE: 0.21903830664184532 	MSE: 43.002151054097546 (clipped to 0)
02GE007 	NSE: 0.3125421162054026 	MSE: 24.86882914855122 (clipped to 0)
02GG002 	NSE: 0.3013673852300638 	MSE: 168.498

In [9]:
writer.close()

In [10]:
load_data.pickle_results('LSTM_VIC', (predictions, actuals), time_stamp)

'LSTM_VIC_20190906-083049.pkl'

In [11]:
datetime.now().strftime('%Y%m%d-%H%M%S')

'20190907-022413'